In [280]:
from music21 import *
from functools import reduce
import numpy
import collections
import functools 
import fractions
import math
import pandas as pd

In [144]:
def fract(x):
    if isinstance(x, fractions.Fraction):
        return x
    else:
        return fractions.Fraction(x)
def fract_gcd(x, y):
    z_top = math.gcd(x.numerator, y.numerator)
    z_bot = x.denominator * y.denominator // math.gcd(x.denominator, y.denominator)
    return fractions.Fraction(z_top, z_bot)
fract_gcd(fractions.Fraction(2, 3), fractions.Fraction(1, 4))

Fraction(1, 12)

In [226]:
def computelast(cells):
    timestamp = set()
    for c in cells:
        start = c.offset
        dur = c.duration.quarterLength
        if start!= 0: timestamp.add(start)
        if dur!= 0: timestamp.add(dur)
    frac_timestamp = [fract(x) for x in timestamp]
    timeunit = reduce(fract_gcd, frac_timestamp)
    return timeunit

def breakcell(score):
    dict_cells = dict()
    current_offset = 0
    finishingtime = 0
    lowest_pitch = highest_pitch = None
    for el in score.recurse():
        if isinstance(el, stream.Measure):
            current_offset =  el.offset
            finishingtime = max(finishingtime, current_offset)
        if isinstance(el, note.Note):
            lowest_pitch = min(lowest_pitch, el.pitches[0].ps) if lowest_pitch is not None else el.pitches[0].ps
            highest_pitch = max(highest_pitch, el.pitches[0].ps) if highest_pitch is not None else el.pitches[0].ps
            if current_offset not in dict_cells:
                dict_cells[current_offset] = [el]
            else:
                dict_cells[current_offset].append(el)
    cell_timeunits = []
    for k,v in dict_cells.items():
        timeunit = computelast(v)
        cell_timeunits.append(timeunit)
    frac_cell_timeunits = [fract(x) for x in cell_timeunits]
    minimum_timeunit =  reduce(fract_gcd, frac_cell_timeunits)     
    print (minimum_timeunit)
    endtime = finishingtime+ max(v.duration.quarterLength for v in dict_cells[finishingtime])
    width = int(highest_pitch - lowest_pitch + 1) 
    assert(fract(endtime) % fract(minimum_timeunit) == 0)
    length = int(fract(endtime) / fract(minimum_timeunit))
    print(width, length)
    m = numpy.zeros(shape=(width,length))
    for k, v in dict_cells.items():
        assert(fract(k)%minimum_timeunit == 0)
        base_l = int(fract(k)/minimum_timeunit)
        for n in v:
            w =int(n.pitches[0].ps - lowest_pitch)
            assert(n.offset%minimum_timeunit == 0)
            l_begin = base_l + int(n.offset/minimum_timeunit)
            assert(n.duration.quarterLength%minimum_timeunit == 0)
            l_end = l_begin+int(n.duration.quarterLength/minimum_timeunit)
            m[w][l_begin:l_end] = 1
    return m
        
        

In [272]:
xml_test = "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/music21/corpus/trecento/PMFC_06_Giovanni-02-Apress_un_fiume_a.xml"
c = converter.parse(xml_test)
#m is the matrix result
m = breakcell(c)

1/12
20 1836
3048


In [228]:
len(c.recurse())

579

In [282]:
#unit test
partial = c.recurse()[0:47]
def compose_score(p):
    currentp = stream.Part(id='part0')
    currentm = None
    for el in p:
        if isinstance(el, stream.Measure):
            if el is not None:
                currentp.append(el)
       
    s = stream.Score(id='mainScore')
    s.insert(0,currentp)
    return s
s = compose_score(partial)

m1 = breakcell(s)
df = pd.DataFrame(m1)
print(df)

1/12
11 153
    0    1    2    3    4    5    6    7    8    9    ...  143  144  145  146  \
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
2   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
3   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0  0.0   
4   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
5   1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  1.0  1.0   
6   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
8   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

    147  148  1